In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x

class BahdanauAttentionNet(nn.Module):
    def __init__(self, in_channels):
        super(BahdanauAttentionNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 8, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels // 8, 1, kernel_size=1)  # Output scalar score for attention
        )

    def forward(self, x):
        return self.fc(x)

class InnerUNet(nn.Module):
    def __init__(self, in_channels=128, features=64):
        super(InnerUNet, self).__init__()
        self.down1 = Block(128, 64, down=True, act="leaky", use_dropout=False)
        self.down2 = Block(64, 128, down=True, act="leaky", use_dropout=False)
        self.down3 = Block(128, 256, down=True, act="leaky", use_dropout=False)
        self.down4 = Block(256, 512, down=True, act="leaky", use_dropout=False)

        self.bottleneck = nn.Sequential(
            nn.Conv2d(512, 512, 4, 2, 1), nn.ReLU()
        )

        self.up1 = Block(512, 256, down=False, act="relu", use_dropout=False)
        self.up2 = Block(256 + 256, 128, down=False, act="relu", use_dropout=False)
        self.up3 = Block(128 + 128, 64, down=False, act="relu", use_dropout=False)
        self.up4 = Block(64 + 64, 256, down=False, act="relu", use_dropout=False)

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        bottleneck = self.bottleneck(d4)
        up1 = self.up1(d4)
        up2 = self.up2(torch.cat([up1, d3], dim=1))
        up3 = self.up3(torch.cat([up2, d2], dim=1))
        up4 = self.up4(torch.cat([up3, d1], dim=1))

        return bottleneck, up2, up4, d4

class OuterUNet(nn.Module):
    def __init__(self, in_channels, features=64):  # Assuming [bs, 3,512,512] input and features == 64
        super(OuterUNet, self).__init__()
        self.down1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )
        self.down2 = Block(64 , 128, down=True, act="leaky", use_dropout=False)
        self.down3 = Block(128, 256, down=True, act="leaky", use_dropout=False)
        self.down4 = Block(256, 512, down=True, act="leaky", use_dropout=False)
        self.down5 = Block(512, 512, down=True, act="leaky", use_dropout=False)
        self.down6 = Block(512, 512, down=True, act="leaky", use_dropout=False)
        self.down7 = nn.Sequential(
            nn.Conv2d(512, 512, 4, 2, 1), nn.ReLU()
        )
        self.inner_unet = InnerUNet(128, 64)

        self.up1 = Block(512 + 512 , 512, down=False, act="relu", use_dropout=True)
        self.up2 = Block(512 + 512, 512, down=False, act="relu", use_dropout=True)
        self.up3 = Block(512 + 512, 512, down=False, act="relu", use_dropout=True)
        self.up4 = Block(512 + 128, 512, down=False, act="relu", use_dropout=False)  # Adjusted input channels to match
        self.up5 = Block(512 + 256, 256, down=False, act="relu", use_dropout=False)
        self.up6 = Block(256 + 256, 128, down=False, act="relu", use_dropout=False)
        self.up7 = Block(128 + 64, 3, down=False, act="relu", use_dropout=False)

        # Bahdanau attention network for h1 and h2
        self.attention_net = BahdanauAttentionNet(in_channels=640)

    def forward(self, x):
        device = x.device  # Ensure we get the device from input tensor

        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)

        bottleneck_inner, up2_inner, up4_inner, d4_inner = self.inner_unet(d2)

        up1 = self.up1(torch.cat([d7, bottleneck_inner], dim=1))
        up2 = self.up2(torch.cat([d4_inner, up1], dim=1))
        up3 = self.up3(torch.cat([up2, d5], dim=1))

        # Move the convolution to the same device as input `d4`
        conv = nn.Conv2d(512, 128, kernel_size=1).to(device)  # Ensure conv is on the same device as d4

        d4_reduced = conv(d4)

        h1 = torch.cat([up3, d4_reduced], dim=1)  #640
        h2 = torch.cat([up3, up2_inner], dim=1)  #640

        e1 = self.attention_net(h1)
        e2 = self.attention_net(h2)

        attention_weights = torch.softmax(torch.cat([e1, e2], dim=1), dim=1)
        a1, a2 = torch.split(attention_weights, 1, dim=1)

        def compute_entropy(attention_map):
            device = attention_map.device

            # Flatten the attention map for the entire batch
            batch_size = attention_map.size(0)  # Get the batch size
            flattened_attention_map = attention_map.view(batch_size, -1)  # Flatten each sample in the batch

            # Apply softmax across the flattened tensor for each sample in the batch
            probs = torch.softmax(flattened_attention_map, dim=1)

            # Compute entropy for each sample in the batch simultaneously
            entropy = -torch.sum(probs * torch.log(probs + 1e-10), dim=1)  # Compute entropy for each sample

            # Aggregate the entropy values (mean or sum)
            aggregated_entropy = torch.mean(entropy)  # Aggregating by mean

            return aggregated_entropy

        entropy_a1 = compute_entropy(a1)
        entropy_a2 = compute_entropy(a2)

        # Compute weighted sum: a1 * d4 + a2 * up2_inner
        weighted_sum = a1 * d4_reduced +  a2 * up2_inner

        # Pass weighted sum to up4
        up4 = self.up4(torch.cat([weighted_sum, up3], dim=1))
        up5 = self.up5(torch.cat([up4, d3], dim=1))
        up6 = self.up6(torch.cat([up4_inner, up5], dim=1))
        up7 = self.up7(torch.cat([up6, d1], dim=1))

        return up7, entropy_a1.item(), entropy_a2.item()


def test_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    x = torch.randn((32, 3, 512, 512)).to(device)
    model = OuterUNet(3, 64).to(device)

    with torch.no_grad():
        preds, e_1, e_2 = model(x)
    print(preds.shape)
    print(e_1)
    print(e_2)

test_model()


torch.Size([32, 3, 512, 512])
6.931302070617676
6.931302547454834


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import os
from PIL import Image
from tqdm import tqdm
from torchvision.utils import save_image
# local module imports
import config
from utils import save_checkpoint, load_checkpoint
from UvU_Net_Generator import OuterUNet as generator
from UvU_Discriminator import Discriminator

torch.backends.cudnn.benchmark = True

In [4]:
def train_fn(
    disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler,
):
    loop = tqdm(loader, leave=True)
    entropy_a1_epoch = 0  # To store accumulated entropy for the epoch
    entropy_a2_epoch = 0

    for idx, (x, y) in enumerate(loop):
        x = x.to(config.DEVICE)
        y = y.to(config.DEVICE)

        # Train Discriminator
        with torch.cuda.amp.autocast():
            print("Shape of X: ", x.shape)
            y_fake, entropy_a1, entropy_a2 = gen(x)  # Generate y_fake with entropy outputs
            print(y_fake.shape, f"Entropy A1: {entropy_a1}, Entropy A2: {entropy_a2}")
            D_real = disc(x, y)
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake = disc(x, y_fake.detach())
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1_loss(y_fake, y) * config.L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        # Accumulate entropy for average calculation
        entropy_a1_epoch += entropy_a1
        entropy_a2_epoch += entropy_a2

        # Print every 10 steps
        if idx % 10 == 0:
            loop.set_postfix(
                D_real=torch.sigmoid(D_real).mean().item(),
                D_fake=torch.sigmoid(D_fake).mean().item(),
            )

    # Calculate and return the average entropy for the epoch
    avg_entropy_a1 = entropy_a1_epoch / len(loader)
    avg_entropy_a2 = entropy_a2_epoch / len(loader)

    return avg_entropy_a1, avg_entropy_a2


In [5]:
!unzip -q Sample_dataset.zip

In [6]:
class PairedImageDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.input_images = sorted(os.listdir(input_dir))
        self.target_images = sorted(os.listdir(target_dir))
        self.transform = transform

        assert len(self.input_images) == len(self.target_images), "Mismatch between input and target images!"

    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):
        input_image_path = os.path.join(self.input_dir, self.input_images[idx])
        target_image_path = os.path.join(self.target_dir, self.target_images[idx])

        input_image = Image.open(input_image_path).convert("RGB")
        target_image = Image.open(target_image_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image


input_dir = '/content/Sample_dataset/sobel_images'
target_dir = '/content/Sample_dataset/input_images'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

dataset = PairedImageDataset(input_dir=input_dir, target_dir=target_dir, transform=transform)

# Split dataset into training and validation (80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Example to iterate over the data loader
for batch_idx, (input_images, target_images) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}")
    print(f"Input batch size: {input_images.shape}")
    print(f"Target batch size: {target_images.shape}")


Batch 1
Input batch size: torch.Size([16, 3, 256, 256])
Target batch size: torch.Size([16, 3, 256, 256])
Batch 2
Input batch size: torch.Size([4, 3, 256, 256])
Target batch size: torch.Size([4, 3, 256, 256])


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Assuming the directory to save checkpoints
save_dir = '/content/drive/MyDrive/model_checkpoints/'  # Change this to your desired directory
def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(config.DEVICE), y.to(config.DEVICE)
    gen.eval()
    with torch.no_grad():
        print("Shape of x at save: ", x.shape)
        y_fake, e1, e2 = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
            save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()

def main():
    disc = Discriminator(in_channels=3).to(config.DEVICE)
    gen = generator(in_channels=3, features=64).to(config.DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    opt_gen = optim.Adam(gen.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    L1_LOSS = nn.L1Loss()

    if config.LOAD_MODEL:
        load_checkpoint(
            config.CHECKPOINT_GEN, gen, opt_gen, config.LEARNING_RATE,
        )
        load_checkpoint(
            config.CHECKPOINT_DISC, disc, opt_disc, config.LEARNING_RATE,
        )

    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()
    for epoch in range(config.NUM_EPOCHS):
        entropy_a1, entropy_a2 = train_fn(
            disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler,
        )

        # Print entropy values
        print(f"Epoch [{epoch + 1}/{config.NUM_EPOCHS}], Avg Entropy A1: {entropy_a1}, Avg Entropy A2: {entropy_a2}")
        # Save models every 50 epochs to Google Drive
        if epoch % 50 == 49:
            print(f"Saving model at epoch {epoch}")
            torch.save({
                'epoch': epoch,
                'generator_state_dict': gen.state_dict(),
                'optimizer_gen_state_dict': opt_gen.state_dict(),
            }, f"{save_dir}generator_epoch_{epoch}.pth")

            torch.save({
                'epoch': epoch,
                'discriminator_state_dict': disc.state_dict(),
                'optimizer_disc_state_dict': opt_disc.state_dict(),
            }, f"{save_dir}discriminator_epoch_{epoch}.pth")

            save_some_examples(gen, val_loader, epoch, folder="evaluation")


if __name__ == "__main__":
    main()


<ipython-input-14-d6a184c76145>:33: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  g_scaler = torch.cuda.amp.GradScaler()
<ipython-input-14-d6a184c76145>:34: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  d_scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-4-591140532064>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-4-591140532064>:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Shape of X:  torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256]) Entropy A1: 5.544980525970459, Entropy A2: 5.544981956481934


100%|██████████| 2/2 [00:00<00:00,  4.15it/s, D_fake=0.422, D_real=0.536]


Shape of X:  torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256]) Entropy A1: 5.5449934005737305, Entropy A2: 5.544992446899414
Epoch [1/500], Avg Entropy A1: 5.544986963272095, Avg Entropy A2: 5.544987201690674


  0%|          | 0/2 [00:00<?, ?it/s]

...

Shape of X:  torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256]) Entropy A1: 5.544950485229492, Entropy A2: 5.544960975646973
Epoch [499/500], Avg Entropy A1: 5.544995307922363, Avg Entropy A2: 5.545003652572632


  0%|          | 0/2 [00:00<?, ?it/s]

Shape of X:  torch.Size([16, 3, 256, 256])
torch.Size([16, 3, 256, 256]) Entropy A1: 5.545068264007568, Entropy A2: 5.545071601867676


100%|██████████| 2/2 [00:00<00:00,  5.28it/s, D_fake=0.00544, D_real=0.997]


Shape of X:  torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256]) Entropy A1: 5.54509162902832, Entropy A2: 5.545094013214111
Epoch [500/500], Avg Entropy A1: 5.545079946517944, Avg Entropy A2: 5.5450828075408936
Saving model at epoch 499
Shape of x at save:  torch.Size([5, 3, 256, 256])


In [ ]:
torch.cuda.empty_cache()


In [ ]:
def main():
    disc = Discriminator(in_channels=3).to(config.DEVICE)
    gen = generator(in_channels=3, features=64).to(config.DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999),)
    opt_gen = optim.Adam(gen.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    L1_LOSS = nn.L1Loss()

    if config.LOAD_MODEL:
        load_checkpoint(
            config.CHECKPOINT_GEN, gen, opt_gen, config.LEARNING_RATE,
        )
        load_checkpoint(
            config.CHECKPOINT_DISC, disc, opt_disc, config.LEARNING_RATE,
        )

    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()

    for epoch in range(config.NUM_EPOCHS):
        train_fn(
            disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler,
        )

        if config.SAVE_MODEL and epoch % 50 == 0:
            save_checkpoint(gen, opt_gen, filename=config.CHECKPOINT_GEN)
            save_checkpoint(disc, opt_disc, filename=config.CHECKPOINT_DISC)
            save_some_examples(gen, val_loader, epoch, folder="evaluation")


if __name__ == "__main__":
    main()

<ipython-input-20-aa02de0e00f1>:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  g_scaler = torch.cuda.amp.GradScaler()
<ipython-input-20-aa02de0e00f1>:18: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  d_scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-13-129499f0dccf>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-13-129499f0dccf>:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
  0%|          | 0/2 [00:00<?, ?it/s]


KeyboardInterrupt: 